# Train YOLO Models in Google Colab
**Author:** Николай Боравлев

**Last updated:**  21/05/2025

**GitHub:** [Обучение модели YOLO и применение для детекции объектов](https://github.com/nboravlev/Trash_detection_with_YOLO)

# Introduction

Этот ноутбук использует фреймворк [Ultralytics](https://docs.ultralytics.com/) для обучения YOLO11, YOLOv8, или YOLOv5 на пользовательском наборе данных. В конце этого ноутбука мы получим готовую модель YOLO, специально обученную для решения задачи детекции на нашем собственном наборе данных, которую можно использовать локально на ПК, телефоне или на внешних устройствах типа Raspberry Pi.



### Working in Colab
Colab предоставляет виртуальную машину, доступную из браузера. В ней уже заранее настроена фаловая система, Python окружение, и самое главное - бесплатные видеокарты для обучения. Скачаем в нашу среду дополнительно PyTorch and Ultralytics, необходимые для обучения, и приступим.







**Verify NVIDIA GPU Availability**

Убедимся, что ноутбук подключен к видеокарте. Для этого необходимо выбрать в меню "Среда выполнения" -> "Сменить среду выполнения" и в разделе "Аппаратный ускоритель" выбрать "Графический процессор". Что убедиться, что подключение установлено, выполним команду:

In [ ]:
!nvidia-smi

Tue May 13 19:05:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# 2.&nbsp;Upload Image Dataset and Prepare Training Data

В этом разделе мы готовим данные для обучения

## 2.1 Upload images

Загрузка изображений

Я скачаю свои данные, которые я предварительно разметил вручную из репозитория GitHub. Архив с данными будет лежать в корневой папке моей текущей работчей среды.

In [1]:
!wget -O /content/data.zip https://github.com/nboravlev/Trash_detection_with_YOLO/raw/main/project_yolo_with_foto_05-13.zip # Trash dataset


--2025-05-24 10:52:40--  https://github.com/nboravlev/Trash_detection_with_YOLO/raw/main/project_yolo_with_foto_05-13.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nboravlev/Trash_detection_with_YOLO/main/project_yolo_with_foto_05-13.zip [following]
--2025-05-24 10:52:41--  https://raw.githubusercontent.com/nboravlev/Trash_detection_with_YOLO/main/project_yolo_with_foto_05-13.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11034622 (11M) [application/zip]
Saving to: ‘/content/data.zip’

/content/data.zip   100%[===================>]  10.52M  51.0MB/s    in 0.2s    

2025-05-24 10:52:41

## 2.2 Split images into train and validation folders

Чтобы разложить данные по папкам для обучения и валидации, необходимо сперва извлечь их из архива. Проверим, что наш архив `data.zip` корректно загружен и теперь лежит в корневой папке в нашей рабочей среде. Далее распакуем архив, выполнив следующий код:

In [2]:
# Unzip images to a custom data folder
!unzip -q /content/data.zip -d /content/custom_data

Ultralytics для обучения модели требуется определенная структура данных: корневая папка называется “data”. Внутри еще две папки:

*   **Train**: В этой папке лежат данные для обучения модели. В течение кажждой эпохи обучения каждое изображение отправляется в модель, и в процессе обучения модель корректирует свои веса с учетом это йинформации.


*   **Validation**: Эти данные не участвуют в обучении, в конце каждой эпохи модель использует эти данные для оценки свое работы.

*   **Test**: Данные для оценки качества работы после обучения. В процессе обучения модель их вообще не будет видеть.

Внутри этих папок данные лежат с папках “images” и “labels”, в которых .содержатся соответственно изображения и аннотации.

Скрит для разделения данных лежит в моем репозитории, сперва необходимо скачать в рабочую среду, затем вызвать, передав в качестве аргументов путь к данным и пару служебных параметров `--train_prc` - доля данных на обучение и `--test_prc`- доля данных на тест. Оставшаяся часть пойдет на валидацию.
Скрипт выполняет сплит и раскладывает данные по папкам в соответсвии со структурой, которую требует Ultralytics для своих моделей.

In [9]:
!wget -O /content/train_val_split.py https://github.com/nboravlev/Trash_detection_with_YOLO/raw/main/Utils/train_val_split.py

# TO DO: Improve robustness of train_val_split.py script so it can handle nested data folders, etc
!python train_val_split.py --datapath="/content/custom_data" --train_pct=.85 --test_pct=.05

--2025-05-24 12:35:07--  https://github.com/nboravlev/Trash_detection_with_YOLO/raw/main/Utils/train_val_split.py
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nboravlev/Trash_detection_with_YOLO/main/Utils/train_val_split.py [following]
--2025-05-24 12:35:07--  https://raw.githubusercontent.com/nboravlev/Trash_detection_with_YOLO/main/Utils/train_val_split.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3669 (3.6K) [text/plain]
Saving to: ‘/content/train_val_split.py’

/content/train_val_ 100%[===================>]   3.58K  --.-KB/s    in 0s      

2025-05-24 12:35:08 (43.0 MB/s) - ‘/conte

# 2.3 Look at the Data

In [7]:
import os

def count_classes(labels_folder):
    class_counts = {}

    # Проход по всем файлам в папке с метками
    for filename in os.listdir(labels_folder):
        if filename.endswith('.txt'):
            file_path = os.path.join(labels_folder, filename)

            with open(file_path, 'r') as file:
                for line in file:
                    line = line.strip()
                    if not line:
                        continue  # Пропускаем пустые строки

                    parts = line.split()
                    if not parts:
                        continue  # Пропускаем строки без данных

                    try:
                        class_id = int(parts[0])
                        class_counts[class_id] = class_counts.get(class_id, 0) + 1
                    except (ValueError, IndexError):
                        continue  # Пропускаем некорректные строки

    # Сортируем классы по возрастанию ID
    sorted_classes = sorted(class_counts.items(), key=lambda x: x[0])

    # Выводим результаты
    print("Количество объектов по классам:")
    for class_id, count in sorted_classes:
        print(f"Класс {class_id}: {count} объектов")
    print(f"Всего классов: {len(sorted_classes)}")
    print(f"Всего объектов: {sum(class_counts.values())}")

# Укажите путь к вашей папке с метками
labels_dir = '/content/custom_data/labels'
count_classes(labels_dir)

Количество объектов по классам:
Класс 0: 454 объектов
Класс 1: 220 объектов
Класс 2: 268 объектов
Всего классов: 3
Всего объектов: 942


# 3.&nbsp;Install Ultralytics

Далее требуется скачать и установить в рабочую среду фреймфорк для обучения и тестирования модели - Ultralytics

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

# 4.&nbsp;Configure Training


Ultralytics для работы требует файл конфигурации YAML. В этом файле прописываются пути к данным и перечисляются классы объектов.
 [Здесь](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/coco128.yaml) можно ознакомиться с примером такого файла.

Код внизу автоматически генерирует `data.yaml` файл. необходимо удостовериться, что карта классов лежит по адресу `custom_data/classes.txt`. Если данные готовились с помощью Label Studio, то такой файлл уже существует. Иначе придется создать файл `classes.txt` самостоятельно в текстовом редакторе и загрузить в Google Colab ([пример](https://github.com/nboravlev/Trash_detection_with_YOLO/blob/main/Docs/classes.txt) файла `classes.txt`).

In [ ]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'test': 'test/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

Created config file at /content/data.yaml

File contents:

path: /content/data
train: train/images
val: validation/images
test: test/images
nc: 3
names:
- bin
- tires
- trash


##5.1 Training Parameters
Данные готовы для обучения, размечены, корректно расположены в тех местах, где их ожидает найти наш скрипт, вспомогательные документы сформированы и загружены в рабочую среду, модель готова обучаться! Перед началом обучения рекумендуется посетить сайт [Ultralytics](https://docs.ultralytics.com/ru/tasks/detect/#models) чтобы ознакомиться с аргументами, которые принимает скрипт запуска обучения. Ниже подробнее остановлюсь на некоторых из них.

**Model architecture & size (`model`):**

Мы всегда ищем баланс между размером модели, стоимостью обучения, качеством и скоростью работы. Модель YOLO11 существует в нескольких размерах yolo11n.pt, yolo11s.pt, yolo11m.pt, yolo11l.pt, and yolo11xl.pt.

![YOLO11](https://github.com/nboravlev/Trash_detection_with_YOLO/raw/main/Docs/YOLO11.PNG)

Логично, что чем больше модель, тем выше качество и ниже скорость работы. Анализируя сравнительную таблицу можно сказать, что yolo11m это довольно хороший выбор для начала.

**Number of epochs (`epochs`)**

"Эпоха" в машинном обучении это один проход модели по всем обучающим данным. От выбора количества эпох зависит продолжительность обучения. На выбор количества эпох влияет, например, количество данных. Если формализовать, то можно сказать, что если количество обучающих изображений меньше 200, то следует брать 60 эпох, если больше, то 40.  После обучения на графике можно будет увидеть по динамике значения **`loss`**, какое количество было бы оптимальным. В любом случае, алгоритм сохраняет версию модели с наилучшими метриками качества.

**Resolution (`imgsz`)**

Этот аргумент отвечает за разрешение изображения, которое передается в модель. Понятно, что от этого параметра напрямую зависит скорость и качество. В целом, стандартом для YOLO является разрешение 640x640, но если заранее известно, что модель будет получать изображения невысокого качества или скорость обучения является приоритетом. можно уменьшить разрешение до, например, 480px

# 5.&nbsp;Train Model

Для инициализации обучения достаточно выполнить блок кода внизу. При необходимости можно подставить свои значения в аргумент `model`, `epochs` или `imgsz`.

In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11l.pt epochs=60 imgsz=640

/bin/bash: line 1: yolo: command not found


Алгоритм обучения парсит изображения из дерикторий \training и \validation и запускает обучение. В конце каждой эпохи алгоритм проходит по данным для валидации и фиксирует метрики mAP, precision, recall. В норме показатель mAP должен возрастать каждую новую эпоху. Цикл обучения повторяется столько раз, сколько указано в аргументе `epochs`.

> **NOTE:** Важно не прерывать цикл обучения. В конце процесса обучения выполнятся технический скрипт-оптимизатор, который оптимизирует размер модели, удаляя ненужные слои и различные артефакты, ненужные для инференса.

Веса модели, которая демонстрирует наилучшие метрики качества, сохраняются в директории `content/runs/detect/train/weights/best.pt`. Дополнительная информация об обучении сохраняется в папку `content/runs/detect/train`. В том числе файл `results.png` с динамикой, как меняется loss, precision, recall, mAP от эпохи к эпохе.

#6.&nbsp;Test Model

Чтобы визуально проверить результат, дадим модели тестовые изображения, которые она не видела на обучении, и посмотрим на результаты предсказания!
Скрипт берет все изображения из папки /test/images, выполняет предсказание и сохраняет картинку с результатом (bbox и класс) в папку `runs/detect/predict. Результат мы потом визуализируем.

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=/content/data/test/images save=True

Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11l summary (fused): 190 layers, 25,281,625 parameters, 0 gradients, 86.6 GFLOPs

image 1/10 /content/data/test/30d66471-326f-434b-9807-8db96ac5429e.jpg: 640x320 1 bin, 103.0ms
image 2/10 /content/data/test/358ba6d3-1470-4afc-ae42-a7db4d26d83d.jpg: 640x320 (no detections), 29.2ms
image 3/10 /content/data/test/IMG_20250505_122812.jpg: 640x480 4 bins, 84.9ms
image 4/10 /content/data/test/IMG_20250506_095157.jpg: 640x480 2 bins, 3 tiress, 36.3ms
image 5/10 /content/data/test/IMG_20250510_155257.jpg: 640x480 3 bins, 2 trashs, 35.8ms
image 6/10 /content/data/test/IMG_20250511_191241.jpg: 640x480 1 bin, 3 trashs, 35.8ms
image 7/10 /content/data/test/IMG_20250512_112119.jpg: 640x480 2 bins, 1 trash, 35.9ms
image 8/10 /content/data/test/IMG_20250512_113545.jpg: 640x480 4 bins, 6 tiress, 35.9ms
image 9/10 /content/data/test/IMG_20250513_194755.jpg: 640x480 1 bin, 4 tiress, 2 trashs, 35.8ms
image 10/10 /conten

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')


In [ ]:
import glob
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# Получаем список путей к изображениям
image_paths = glob.glob('/content/runs/detect/predict/*.jpg')#[:10] если хотим вывести первые Х файлов

# Настройки отображения
images_per_row = 4
total_images = len(image_paths)
rows = (total_images // images_per_row) + (1 if total_images % images_per_row > 0 else 0)

# Рисуем
fig, axes = plt.subplots(rows, images_per_row, figsize=(15, rows * 4))
axes = axes.flatten()

for ax, image_path in zip(axes, image_paths):
    img = plt.imread(image_path)
    ax.imshow(img)
    ax.axis('off')

# Скрываем пустые subplot'ы
for ax in axes[total_images:]:
    ax.axis('off')

plt.tight_layout()
plt.show()

Модель обводит `bbox` вокруг каждого интересующего нас объекта. Если обводит некорректно, то вот короткий список вещей, на которые следует обратить внимание:


1. В первую очередь это конечно качество данных: ошибки и всякого рода противоречия в разметке. Воообще качество данных это самая чуствительная и важная часть. `Что положишь в модель, то и получишь`
2. Увеличить количество эпох. Опять же, смотря по динамике loss: если loss вышел на плато, едва ли это поможет.
3. Взять модель побольше (e.g. `yolo11l.pt`).
4. Добавить примеров в обучающий датасет. Хорошее [видео](https://www.youtube.com/watch?v=v0ssiOY6cfg) о том, как самому выполнить разметку.

Можно также запустить модель на других данных: изображениях или видео. Для этого необходимо загрузить данные в текущую рабочую среду и выполнить `!yolo detect predict`, указав источник данных в `source`. Результаты сохраняются в папке `runs/detect/predict`.

Это все весьма интересно, однако понятно, что использовать модель в ноутбуке Google Colab большого практического смысла нет. Пришла пора забирать отсюда обученную модель, интегрировать ее в приложение или на сайт для решения важных практических задач.


#7.&nbsp;Deploy Model

Модели YOLO можно запускать на разлиных девайсах: PC, телефон, специальные автономные системы (Raspberry Pi, смарткамеры, беспилотники - часто, в условиях жесткого ограничения по ресурсам). Ultralytics позволяет конвертировать модель в другие форматы (`tflite`, `onnx`, etc.) для использования в различных средах.


## 7.1 Download YOLO Model

Здесь архивируем и сохраняем свой экземпляр модели.

Запуск этого блока создает папку `my_model`, сохраняет в нее архив, в котором лежит версия `best.pt` (переименованная в `my_model.pt`) и дополительная информация о процессе обучения. Архив будет называться `my_model.zip`.

In [ ]:
# Create "my_model" folder to store model weights and train results
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

# Zip into "my_model.zip"
%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

In [ ]:
# Можно выполнить код, чтобы сохранить архив локально и просто пойти в сайдбар и скачать.
from google.colab import files

files.download('/content/my_model.zip')

## 7.2 Deploy YOLO Model on Local Devices

На этом этапе мы заканчиваем работу в данном ноутбуке. Процесс деплоя модели в приложение для детекции объектов будет развернут в моем [репозитории](https://github.com/nboravlev/Trash_detection_with_YOLO/tree/main/App) на GitHub